# reference:
- https://github.com/jjone36/Cosmetic/blob/master/01.scraping.py

In [1]:
import time

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver import ChromeOptions

In [8]:
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook as tqdm
from datetime import datetime
from collections import OrderedDict
import pandas as pd
import requests
import pickle
import re

In [ ]:
#TEST: first product and exit signin box
chrome_path = "/Users/ChristieFung/Desktop/Insight/skin_care_scraper-2/chromedriver"


option = webdriver.ChromeOptions()
chrome_prefs = {}
option.experimental_options["prefs"] = chrome_prefs

chrome_prefs["profile.default_content_settings"] = { "popups": 1 }

driver = webdriver.Chrome(executable_path = chrome_path, chrome_options=option)

xpath = '//body[@class="css-1rq8sta"]//div[contains(@class,"css-111ktzd")]//div//div//div[1]//div[1]//a[1]//div[1]//div[4]//span[1]'
btn = driver.find_element_by_xpath(xpath)
btn.click()
time.sleep(10)
    
xpath2 = '//div[@class="css-dch3kx"]//button[contains(@class,"css-1rq477f")]' #exit signin box
btn = driver.find_element_by_xpath(xpath2)
btn.click()


##  only get the label, url, first review of each product, ?? skin types, no name of product, no ratings# 

In [99]:
chrome_path = "/Users/ChristieFung/Desktop/Insight/skin_care_scraper-2/chromedriver"

def scrollDown(driver, n_scroll):
    body = driver.find_element_by_tag_name("body")
    while n_scroll >= 0:
        body.send_keys(Keys.PAGE_DOWN)
        n_scroll -= 1
    return driver

option = webdriver.ChromeOptions()
chrome_prefs = {}
option.experimental_options["prefs"] = chrome_prefs

chrome_prefs["profile.default_content_settings"] = { "popups": 1 }

driver = webdriver.Chrome(executable_path = chrome_path, chrome_options=option)

df = pd.DataFrame(columns=['Label', 'URL'])
print(df)

tickers = ['acne-treatment-blemish-remover', 'dry-skin-treatment', 'pore-minimizing-products', 'moisturizer-skincare']

for ticker in tickers:
    url = 'https://www.sephora.com/shop/' + ticker + '?pageSize=300'
    driver.get(url)
    
    browser = scrollDown(driver, 10)
    time.sleep(10)

    browser = scrollDown(driver, 10)
    time.sleep(10)

    element = driver.find_elements_by_class_name('css-ix8km1')

    subpageURL = []
    for a in element:
        subURL = a.get_attribute('href')
        subpageURL.append(subURL)

    # transform into a data frame
    dic = {'Label': ticker, 'URL': subpageURL}
    df = df.append(pd.DataFrame(dic), ignore_index = True)

# add columns
df2 = pd.DataFrame(columns=['brand', 'name', 'price', 'ratings', 'reviews', 'skin_type'])
df = pd.concat([df, df2], axis = 1)
    

for i in range(len(df)+1):
    url = df.URL[i]
    driver.get(url)
    time.sleep(5)

    # brand, name, price
    
    df.brand[i] = driver.find_element_by_class_name('css-euydo4').text
    df.name[i] = driver.find_element_by_class_name('css-0 span').text
    df.price[i] = driver.find_element_by_class_name('css-slwsq8').text   

    browser = scrollDown(driver, 1)
    time.sleep(2)
    browser = scrollDown(driver, 1)
    time.sleep(2)
    
    xpath = '//button[contains(@class,"css-frqcui")]'
    btn = driver.find_element_by_xpath(xpath)
    btn.click()
    
    df.ratings[i] = driver.find_element_by_class_name('css-3z5ot7').text
    df.reviews[i] = driver.find_element_by_class_name('css-7rv8g1').text
    df.skin_type[i]= driver.find_element_by_class_name('css-10lg0rx ').text

    print(i)    # just for verbose
    
df.to_csv('data/cosmetic.csv', encoding = 'utf-8-sig', index = False)

Empty DataFrame
Columns: [Label, URL]
Index: []
0
1
2


KeyboardInterrupt: 

In [100]:
df

,Label,URL,brand,name,price,ratings,reviews,skin_type
0,acne-treatment-blemish-remover,https://www.sephora.com/product/c-firma-day-se...,DRUNK ELEPHANT,,$80.00,,Got this as a 100 point reward and while okay ...,Age 25-34
1,acne-treatment-blemish-remover,https://www.sephora.com/product/soy-face-clean...,FRESH,,$38.00,,This cleanser immediately leaves my skin feeli...,Eye Color Blue
2,acne-treatment-blemish-remover,https://www.sephora.com/product/t-l-c-framboos...,DRUNK ELEPHANT,,$90.00,,This product is a miracle I can no longer live...,Eye Color Green
3,acne-treatment-blemish-remover,https://www.sephora.com/product/the-ordinary-d...,NaN,NaN,NaN,NaN,NaN,NaN
4,acne-treatment-blemish-remover,https://www.sephora.com/product/t-l-c-sukari-b...,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
427,moisturizer-skincare,https://www.sephora.com/product/purity-made-si...,NaN,NaN,NaN,NaN,NaN,NaN
428,moisturizer-skincare,https://www.sephora.com/product/photo-plasma-m...,NaN,NaN,NaN,NaN,NaN,NaN
429,moisturizer-skincare,https://www.sephora.com/product/face-finishing...,NaN,NaN,NaN,NaN,NaN,NaN
430,moisturizer-skincare,https://www.sephora.com/product/5-in-1-face-cr...,NaN,NaN,NaN,NaN,NaN,NaN


In [103]:
df.iloc[0]

Label                           acne-treatment-blemish-remover
URL          https://www.sephora.com/product/c-firma-day-se...
brand                                           DRUNK ELEPHANT
name                                                          
price                                                   $80.00
ratings                                                       
reviews      Got this as a 100 point reward and while okay ...
skin_type                                            Age 25-34
Name: 0, dtype: object

# only get a list of skin types# 

In [65]:

chrome_path = "/Users/ChristieFung/Desktop/Insight/skin_care_scraper-2/chromedriver"

def scrollDown(driver, n_scroll):
    body = driver.find_element_by_tag_name("body")
    while n_scroll >= 0:
        body.send_keys(Keys.PAGE_DOWN)
        n_scroll -= 1
    return driver

option = webdriver.ChromeOptions()
chrome_prefs = {}
option.experimental_options["prefs"] = chrome_prefs

chrome_prefs["profile.default_content_settings"] = { "popups": 1 }

driver = webdriver.Chrome(executable_path = chrome_path, chrome_options=option)

df = pd.DataFrame(columns=['Label', 'URL'])
print(df)

# step 1
#tickers = ['acne-treatment-blemish-remover', 'dry-skin-treatment', 'pore-minimizing-products', 'moisturizer-skincare']

#tickers = ['acne-treatment-blemish-remover']


url = 'https://www.sephora.com/product/soy-face-cleanser-P7880?icid2=products%20grid:p7880'
driver.get(url)

#    xpath = '//div[@class="css-dkxsdo"]//div[1]//a[1]//div[1]//div[4]'
#    btn = driver.find_element_by_xpath(xpath)
#    btn.click()
#    time.sleep(20)

browser = scrollDown(driver, 10)
time.sleep(10)

browser = scrollDown(driver, 10)
time.sleep(10)

element = driver.find_elements_by_class_name('css-ix8km1')

fields = driver.find_elements_by_class_name('css-j5yt83')
for i, field in enumerate(fields):
    print(i, field.text)

Empty DataFrame
Columns: [Label, URL]
Index: []
0 
1 
2 Eye Color Blue
Hair color Blonde
Skin Tone Light
Skin Type Normal
3 Eye Color Green
Hair color Blonde
Skin Tone Fair
Skin Type Combination
4 Eye Color Brown
Hair color Auburn
Skin Tone Light
Skin Type Combination
5 Eye Color Brown
Hair color Black
Skin Tone Medium
Skin Type Combination
6 Hair color Brunette
Skin Tone Light
Skin Type Oily
7 Eye Color Green
Hair color Blonde
Skin Tone Fair
Skin Type Dry


# Original code

In [58]:
import time

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException


chrome_path = "C:\\Users\jjone\Downloads\chromedriver"

def scrollDown(driver, n_scroll):
    body = driver.find_element_by_tag_name("body")
    while n_scroll >= 0:
        body.send_keys(Keys.PAGE_DOWN)
        n_scroll -= 1
    return driver


driver = webdriver.Chrome(executable_path = chrome_path)

url = 'https://www.sephora.com'
driver.get(url)

xpath = '/html/body/div[5]/div/div/div[1]/div/div/button'
btn = driver.find_element_by_xpath(xpath)
btn.click()
xpath2 = '/html/body/div[3]/div/div/div[1]/div/div/div[2]/form/div[3]/div/div[1]/button'
btn = driver.find_element_by_xpath(xpath2)
btn.click()

# initiate empty dataframe
df = pd.DataFrame(columns=['Label', 'URL'])
print(df)

# step 1
tickers = ['moisturizing-cream-oils-mists', 'cleanser', 'facial-treatments', 'face-mask',
           'eye-treatment-dark-circle-treatment', 'sunscreen-sun-protection']

for ticker in tickers:
    url = 'https://www.sephora.com/shop/' + ticker + '?pageSize=300'
    driver.get(url)

    xpath = '/html/body/div[5]/div/div/div[1]/div/div/button'
    btn = driver.find_element_by_xpath(xpath)
    btn.click()
    time.sleep(20)

    browser = scrollDown(driver, 10)
    time.sleep(10)

    browser = scrollDown(driver, 10)
    time.sleep(10)

    browser = scrollDown(driver, 10)
    time.sleep(10)

    browser = scrollDown(driver, 10)

    element = driver.find_elements_by_class_name('css-ix8km1')

    subpageURL = []
    for a in element:
        subURL = a.get_attribute('href')
        subpageURL.append(subURL)

    # transform into a data frame
    dic = {'Label': ticker, 'URL': subpageURL}
    df = df.append(pd.DataFrame(dic), ignore_index = True)

# add columns
df2 = pd.DataFrame(columns=['brand', 'name', 'price', 'rank', 'skin_type', 'ingredients'])
df = pd.concat([df, df2], axis = 1)

# step 2
for i in range(len(df)+1):
    url = df.URL[i]
    driver.get(url)
    time.sleep(5)

    xpath = '/html/body/div[5]/div/div/div[1]/div/div/button'
    btn = driver.find_element_by_xpath(xpath)
    btn.click()

    # brand, name, price
    df.brand[i] = driver.find_element_by_class_name('css-avdj50').text
    df.name[i] = driver.find_element_by_class_name('css-r4ddnb ').text
    df.price[i] = driver.find_element_by_class_name('css-n8yjg7 ').text

    browser = scrollDown(driver, 1)
    time.sleep(5)
    browser = scrollDown(driver, 1)
    time.sleep(5)

    # skin_type
    detail = driver.find_element_by_class_name('css-192qj50').text
    pattern = r"✔ \w+\n"
    df.skin_type[i] = re.findall(pattern, detail)

    # ingredients
    xpath = '//*[@id="tab2"]'
    btn = driver.find_element_by_xpath(xpath)
    btn.click()

    try:
        df.ingredients[i] = driver.find_element_by_xpath('//*[@id="tabpanel2"]/div').text
    except NoSuchElementException:
        df.ingredients[i] = 'No Info'

    # rank
    try:
        rank = driver.find_element_by_class_name('css-ffj77u').text
        rank = re.match('\d.\d', rank).group()
        df['rank'][i] = str(rank)

    except NoSuchElementException:
        df['rank'][i] = 0

    print(i)    # just for verbose


df.to_csv('data/cosmetic.csv', encoding = 'utf-8-sig', index = False)

Empty DataFrame
Columns: [Label, URL]
Index: []


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class="css-dch3kx"]//button[contains(@class,"css-1rq477f")]"}
  (Session info: chrome=83.0.4103.97)
